In [3]:
import face_recognition as fr
import matplotlib.pyplot as plt
import cv2 as cv
import os
import requests
import numpy as np
Unique_Faces_encoding = {}
person_count = 0

meta_data ={
}

In [4]:
def generate_key(n):
    result = ""
    while n > 0:
        n -= 1  # Decrease n by 1 to make modulo operation compatible with 'A' being 1 not 0
        remainder = n % 26
        result = chr(65 + remainder) + result
        n = n // 26
    return result

def save_img(img,counter):
    if not os.path.exists('Unique Faces'):
        os.makedirs('Unique Faces')
    
    file_name = generate_key(counter)
    path = "Unique Faces/"+file_name+".jpg"
    img_bgr = cv.cvtColor(img,cv.COLOR_RGB2BGR)
    cv.imwrite(path,img_bgr)
    
    return file_name

def read_image_from_url(url):
    try:
        # Send a GET request to the URL to fetch the image
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Convert the response content to a numpy array
            image_array = np.frombuffer(response.content, np.uint8)
            # Decode the image array using OpenCV
            image = cv.imdecode(image_array, cv.IMREAD_COLOR)
            image_rgb = cv.cvtColor(image,cv.COLOR_BGR2RGB)
            return image_rgb
        else:
            print("Failed to retrieve image. Status code:", response.status_code)
            return None
    except Exception as e:
        print("Error:", e)
        return None

def is_unique(img_enc):

    for face_enc in Unique_Faces_encoding:
        known_face = np.array([Unique_Faces_encoding[face_enc]])
        un_known_face = img_enc    

        dis = fr.face_distance(known_face, un_known_face)
        # print(dis)
        if dis < 0.35:
            return (False,face_enc)
    
    return (True,-1)


In [5]:
def find_faces(file_name,img):
    global person_count
    in_persons = []
    face_locations = fr.face_locations(img)
    for face in face_locations:
        top, right, bottom, left = face
        crp_face = img[top:bottom, left:right]
        crp_enc = fr.face_encodings(img,[face])[0]

        unique , key = is_unique(crp_enc)
        if unique:
            person_count += 1
            key=save_img(crp_face,person_count)
            Unique_Faces_encoding[key] = crp_enc
            
        
        in_persons.append(key)
        if key not in meta_data:
            meta_data[key]= []
        
        meta_data[key].append(file_name)
            
    print(person_count,in_persons)

In [1]:
import cloudinary
import os
import cloudinary.uploader
import cloudinary.api
cloudinary.config( 
  cloud_name = "dn5fjw67n", 
  api_key = "563745582142893", 
  api_secret = "GqWGqqcGBwwia5xNbxI-g9oxSkw",
  secure = True
)

In [20]:
def save_img1(filename,img,counter):

    key = generate_key(counter)
    success, encoded_image = cv.imencode('.jpg', img)
    if success:
    # Convert the encoded image data to bytes and upload to Cloudinary
        img_data = encoded_image.tobytes()
        path = filename+"/Unique_Faces"
        cloudinary.uploader.upload_image(img_data,folder = path,public_id = key)
    # return key


In [21]:
img = cv.imread(r"D:\HelloSnaps\notebooks\Unique Faces\B.jpg")

In [22]:
save_img1('TestFolder',img,1)

In [ ]:
success, encoded_image = cv.imencode('.jpg', img_bgr)
if success:
    # Convert the encoded image data to bytes and upload to Cloudinary
    image_data = encoded_image.tobytes()